In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import  matplotlib.pyplot as plt 
from pathlib import Path

# sns.color_palette("colorblind")
# sns.set_palette(sns.color_palette("colorblind"))
sns.set_palette(sns.color_palette("muted"))

pd.set_option('display.max_columns', None)

Сущности базы данных:  
- contact_preference — справочник предпочтительного способа связи;  
- customer_preferenced_contact_type — таблица связи пользователей(предпочтительный способ связи);  
- customer — справочник пользователей с основной информацией по ним;  
- customers_segmetns — таблица связей пользователей и сегментов;  
- location — справочник локаций;  
- product_feedback — отзывы по продуктам;  
- product — справочник продуктов с именем, категорией, ценой;  
- purchase — таблица покупок;  
- segment — справочник сегментов с информацией о названии сегмента и скидки;  
- service_feedback — отзывы о сервисе  

Задача 1: собрать всю информацию по каждому пользователю в отдельной  
витрине, а также на основе имеющихся данных посчитать:  
- количество покупок,  
- количество потраченных средств,  
- средний рейтинг по продукту,  
- наивысший продукт по рейтингу,  
- самый популярный тип продукта,  
- количество отзывов.  

In [2]:
db_dir = './datasets/hw_7_db/'

def ov_read_csv(folder_path: str, file: str) -> pd.DataFrame:
    return pd.read_csv(Path(db_dir) / Path(file), encoding='utf-8', sep=';')

In [3]:
# можно было словарем, но обращаться каждый раз через [''] или get - отстой
contact_preference = ov_read_csv(db_dir, 'contact_preference.csv')
customer_preferenced_contact_type =  ov_read_csv(db_dir, 'contact_preference.csv')
customer = ov_read_csv(db_dir, 'customer.csv')
customers_segmetns =  ov_read_csv(db_dir, 'customers_segmetns.csv')
contact_preference = ov_read_csv(db_dir, 'contact_preference.csv')
location =  ov_read_csv(db_dir, 'location.csv')
product = ov_read_csv(db_dir, 'product.csv')
purchase =  ov_read_csv(db_dir, 'purchase.csv')
segment = ov_read_csv(db_dir, 'segment.csv')
service_feedback =  ov_read_csv(db_dir, 'service_feedback.csv')
product_feedback = ov_read_csv(db_dir, 'product_feedback.csv')

In [4]:
customer.columns

Index(['id', 'name', 'email', 'location_id', 'registration_date'], dtype='object')

In [5]:
customers_segmetns.columns

Index(['customer_id', 'segment_id'], dtype='object')

In [6]:
segment.columns

Index(['id', 'segment_name', 'discount'], dtype='object')

In [7]:
service_feedback.columns

Index(['id', 'customer_id', 'rating', 'comment'], dtype='object')

In [8]:
product_feedback.columns

Index(['id', 'purchase_id', 'rating', 'review'], dtype='object')

In [9]:
purchase.columns

Index(['id', 'user_id', 'product_id', 'purchase_date', 'quantity'], dtype='object')

In [10]:
purchase

,id,user_id,product_id,purchase_date,quantity
0,293813371,19,104,2024-02-09,2
1,696524227,49,90,2023-06-30,1
2,14417739,30,20,2023-09-28,1
3,539614646,45,141,2023-04-25,2
4,691165168,40,25,2023-10-26,2
...,...,...,...,...,...
495,610418601,24,66,2023-07-19,2
496,567876336,46,75,2023-08-31,1
497,956349124,9,111,2023-06-04,2
498,83968957,9,143,2024-02-18,1


In [26]:
df = (customer.merge(customers_segmetns, 'left', left_on='id', right_on='customer_id', suffixes=('_x', '_y'))
         .merge(segment, 'left', left_on='segment_id', right_on='id', suffixes=('_sx', '_sy'))
         .merge(service_feedback, 'left', left_on='customer_id', right_on='customer_id', suffixes=('_sfx', '_sfy'))
         .merge(purchase, 'left', left_on='customer_id', right_on='user_id', suffixes=('_px', '_py'))
         .merge(product, 'left', left_on='product_id', right_on='id', suffixes=('_px2','_prod_id'))
         .merge(product_feedback, 'left', left_on='id_py', right_on=('purchase_id'), suffixes=('_servicefeed','_prodfeedy'))
)
df

,id_sx,name,email,location_id,registration_date,customer_id,segment_id,id_sy,segment_name,discount,id_px,rating_servicefeed,comment,id_py,user_id,product_id,purchase_date,quantity,id_servicefeed,product_name,category_name,price,id_prodfeedy,purchase_id,rating_prodfeedy,review
0,1,Alice,alice@example.com,1,2023-01-15,1,2,2,Regular,3,12.0,7.0,"Satisfactory service, nothing exceptional.",618102097.0,1.0,79.0,2024-01-28,1.0,79.0,Journey,Games,14.99,94.0,618102097.0,5.0,Quality is lower than expected.
1,1,Alice,alice@example.com,1,2023-01-15,1,2,2,Regular,3,12.0,7.0,"Satisfactory service, nothing exceptional.",3228114.0,1.0,53.0,2024-01-22,1.0,53.0,Call of Duty,Games,59.99,155.0,3228114.0,8.0,Very happy with my order!
2,1,Alice,alice@example.com,1,2023-01-15,1,2,2,Regular,3,12.0,7.0,"Satisfactory service, nothing exceptional.",54617801.0,1.0,50.0,2024-01-12,1.0,50.0,The Art of War,Books,7.99,185.0,54617801.0,9.0,Great quality and service!
3,1,Alice,alice@example.com,1,2023-01-15,1,2,2,Regular,3,12.0,7.0,"Satisfactory service, nothing exceptional.",38395521.0,1.0,34.0,2024-01-20,1.0,34.0,Life of Pi,Books,10.99,196.0,38395521.0,10.0,Love this product!
4,1,Alice,alice@example.com,1,2023-01-15,1,2,2,Regular,3,12.0,7.0,"Satisfactory service, nothing exceptional.",32847873.0,1.0,107.0,2023-09-11,2.0,107.0,The Shawshank Redemption,Movies,14.99,202.0,32847873.0,6.0,Quality is subpar.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,49,Walter,walter@example.com,9,2023-05-25,49,1,1,New,2,NaN,NaN,NaN,606726160.0,49.0,86.0,2023-10-10,2.0,86.0,Mario Kart 8 Deluxe,Games,59.99,240.0,606726160.0,10.0,"Excellent, will buy again!"
497,49,Walter,walter@example.com,9,2023-05-25,49,1,1,New,2,NaN,NaN,NaN,739220618.0,49.0,107.0,2023-11-25,1.0,107.0,The Shawshank Redemption,Movies,14.99,NaN,NaN,NaN,NaN
498,49,Walter,walter@example.com,9,2023-05-25,49,1,1,New,2,NaN,NaN,NaN,315015784.0,49.0,89.0,2023-09-19,1.0,89.0,Gears of War 5,Games,39.99,NaN,NaN,NaN,NaN
499,49,Walter,walter@example.com,9,2023-05-25,49,1,1,New,2,NaN,NaN,NaN,54540276.0,49.0,31.0,2023-11-06,2.0,31.0,The Fault in Our Stars,Books,9.99,NaN,NaN,NaN,NaN


,id,purchase_id,rating,review


## 1.1 Datamart по каждому пользователю
- количество покупок,  
- количество потраченных средств,  
- средний рейтинг по продукту,  - не продуктам? так витрина будет уже о клиентах - продуктах, ну ок
- наивысший продукт по рейтингу,  
- самый популярный тип продукта,  
- количество отзывов.  

In [15]:
# df.head(10)

In [38]:
df.tail()

,id_sx,name,email,location_id,registration_date,customer_id,segment_id,id_sy,segment_name,discount,id_px,rating_servicefeed,comment,id_py,user_id,product_id,purchase_date,quantity,id_servicefeed,product_name,category_name,price,id_prodfeedy,purchase_id,rating_prodfeedy,review
496,49,Walter,walter@example.com,9,2023-05-25,49,1,1,New,2,NaN,NaN,NaN,606726160.0,49.0,86.0,2023-10-10,2.0,86.0,Mario Kart 8 Deluxe,Games,59.99,240.0,606726160.0,10.0,"Excellent, will buy again!"
497,49,Walter,walter@example.com,9,2023-05-25,49,1,1,New,2,NaN,NaN,NaN,739220618.0,49.0,107.0,2023-11-25,1.0,107.0,The Shawshank Redemption,Movies,14.99,NaN,NaN,NaN,NaN
498,49,Walter,walter@example.com,9,2023-05-25,49,1,1,New,2,NaN,NaN,NaN,315015784.0,49.0,89.0,2023-09-19,1.0,89.0,Gears of War 5,Games,39.99,NaN,NaN,NaN,NaN
499,49,Walter,walter@example.com,9,2023-05-25,49,1,1,New,2,NaN,NaN,NaN,54540276.0,49.0,31.0,2023-11-06,2.0,31.0,The Fault in Our Stars,Books,9.99,NaN,NaN,NaN,NaN
500,50,Xena,xena@example.com,10,2023-05-30,50,2,2,Regular,3,10.0,10.0,Best service I've ever received!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
# я не понял как действует параметр discount, поэтому по потраченным средствам предпологаю что заплачено с учетом скидки
datamart = df.groupby(['customer_id'])\
             .agg({'rating_prodfeedy': 'mean',
                   'review': 'count'}).reset_index()
datamart = datamart.merge(df.groupby('customer_id')
                  .agg({'rating_prodfeedy': 'max'}).reset_index(), 'inner', on='customer_id')

In [160]:
datamart.rename(columns={'rating_prodfeedy_x': 'mean_rating', 
                         'review': 'review_cnt',
                        'rating_prodfeedy_y': 'max_rating'}, inplace=True)

In [161]:
datamart = datamart.merge(df['customer_id'].value_counts().reset_index().rename(columns={'count': 'purchase_cnt'}),
                          'inner',  on='customer_id')

In [162]:
sample_df = df.groupby(['customer_id', 'product_id'])\
  .agg({'product_id': 'count'}).rename(columns={'product_id': 'cnt'}).reset_index(level=[0,1])

In [163]:
# самый популярный тип продукта
datamart = datamart.merge(sample_df.sort_values('cnt', ascending=False).groupby(['customer_id']).head(1).rename(columns={'product_id': 'fav_product_id'}).drop('cnt', axis=1),
               'inner', on='customer_id')
datamart.head()

,customer_id,mean_rating,review_cnt,max_rating,purchase_cnt,fav_product_id
0,1,8.000,6,10.0,11,71.0
1,2,8.125,8,10.0,10,110.0
2,3,8.750,4,10.0,10,132.0
3,4,7.625,8,10.0,14,12.0
4,5,9.000,3,9.0,9,22.0


In [164]:
# количество потраченных средств
df['payed'] = df['quantity'] * df['price'].fillna(0).replace('Free', 0).astype(float)

In [165]:
total_spent = df.groupby('customer_id')\
  .agg({'payed':'sum'}).reset_index().rename(columns={'payed':'total_spent'})
total_spent.head()

,customer_id,total_spent
0,1,563.84
1,2,398.34
2,3,247.85
3,4,359.27
4,5,164.91


In [166]:
datamart = datamart.merge(total_spent, 'inner', on='customer_id')\
                   .merge(product[['id', 'product_name']], 'inner', left_on='fav_product_id', right_on='id')\
                   .drop(['fav_product_id', 'id'], axis=1)\
                   .rename(columns={'product_name':'favorite_product'})
# datamart = datamart['customer_id', 'mean_rating', 're'


In [167]:
datamart

,customer_id,mean_rating,review_cnt,max_rating,purchase_cnt,total_spent,favorite_product
0,1,8.000000,6,10.0,11,563.84,Animal Crossing: New Horizons
1,22,8.000000,2,8.0,10,432.86,Animal Crossing: New Horizons
2,2,8.125000,8,10.0,10,398.34,Jurassic Park
3,3,8.750000,4,10.0,10,247.85,Coco
4,27,8.250000,4,10.0,9,266.36,Coco
5,4,7.625000,8,10.0,14,359.27,Brave New World
6,21,8.600000,5,10.0,10,247.86,Brave New World
7,5,9.000000,3,9.0,9,164.91,The Fault in Our Stars
8,6,6.750000,4,8.0,7,478.39,Goodfellas
9,7,7.500000,4,10.0,7,161.89,Get Out


## Задача 2: 
написать функцию, которая считает долю трат пользователя по разным  
категориям от общих трат пользователя. Категории: Movies, Games, Books.  


In [203]:
def category_ration_cnt(df):
    cat_payed = df.groupby(['customer_id', 'category_name'])\
              .agg({'payed': 'sum'})\
              .reset_index(level=[0,1])
    sts_pivot = pd.pivot_table(cat_payed, index='customer_id', values='payed', columns='category_name').reset_index(level=[0]).fillna(0)
    sts_pivot['total'] = sts_pivot['Books'] + sts_pivot['Games'] + sts_pivot['Movies']
    
    for cat in ['Books', 'Games', 'Movies']:
        sts_pivot[f'{cat}_of_total_rt'] = round(sts_pivot[cat] / sts_pivot['total'], 2)
    return sts_pivot[['customer_id', 'Books_of_total_rt', 'Games_of_total_rt', 'Movies_of_total_rt']].set_index('customer_id')

In [204]:
category_ration_cnt(df)

category_name,Books_of_total_rt,Games_of_total_rt,Movies_of_total_rt
customer_id,,,
1,0.07,0.88,0.05
2,0.12,0.63,0.25
3,0.33,0.24,0.43
4,0.22,0.46,0.32
5,0.33,0.58,0.09
6,0.03,0.94,0.03
7,0.30,0.19,0.51
8,0.23,0.67,0.10
9,0.25,0.27,0.49
